In [42]:
import cv2
import argparse

from ultralytics import YOLO
import supervision as sv
import numpy as np
import math
import json
import random 

SAVE_IMG = True

In [43]:
def distance(x1, y1, x2, y2):
    """
    Calculate the distance between two points in 2D space using the Euclidean distance formula.

    Parameters:
    x1 (float): x-coordinate of point 1
    y1 (float): y-coordinate of point 1
    x2 (float): x-coordinate of point 2
    y2 (float): y-coordinate of point 2

    Returns:
    float: The distance between the two points.
    """
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def add_trace(round, category, dict_traces, current_pos):
    if round not in dict_traces:
        dict_traces[round] = {'frames': [], 'complete_trace': [],'before_bounce': [], 'after_bounce': [], 'after_bat': [], 
                              'bounce_point': [],'bounce_point_proj': [], 'bounce_d_meters': [],'speed': [],
                              'miss_point': [],'miss_point_proj': []}
    
    if category not in dict_traces[round]:
        dict_traces[round][category] = []
    
    dict_traces[round][category].append(current_pos)

def interpolate_trajectory(trajectory, max_distance):
    interpolated_trajectory = []
    
    for i in range(len(trajectory) - 1):
        x1, y1 = trajectory[i]
        x2, y2 = trajectory[i + 1]
        
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        
        if distance > max_distance:
            num_points = int(distance // max_distance) + 1
            dx = (x2 - x1) / num_points
            dy = (y2 - y1) / num_points
            
            for j in range(num_points):
                x = x1 + j * dx
                y = y1 + j * dy
                interpolated_trajectory.append([x, y])
        
        interpolated_trajectory.append([x2, y2])
    
    return interpolated_trajectory

In [44]:
# List of ball xy positions (all and current trace)
traces = {}

bounced = False
batted = False
missed = False

n_frame = 0


########## PROJECTION OF WICKETS #############

x0_batting = 25
y0_batting = 150

batting_area = [[ 411,  524],
                [411, 150],
                [955, 142],
                [ 955,  514]]

# New wickets
wickets = [[[660, 509], [660, 372], [671, 373],[671, 509]],
            [[680, 509], [680, 372], [689, 372],[689, 509]],
            [[698, 509], [698, 369], [707, 370],[707, 509]]]

w_batting_area = batting_area[3][0] - batting_area[1][0]
h_batting_area = batting_area[0][1] - batting_area[1][1]

w_batting_proj = 100
h_batting_proj = int(h_batting_area*(w_batting_proj/w_batting_area))


batting_proj =  [[x0_batting,y0_batting+h_batting_proj], 
                [x0_batting,y0_batting], 
                [x0_batting+w_batting_proj,y0_batting], 
                [x0_batting+w_batting_proj,y0_batting+h_batting_proj]]

# Perspective transformation
matrix_wickets = cv2.getPerspectiveTransform(np.float32(batting_area), np.float32(batting_proj))

wickets_transformed =[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]

# Loop over each wicket
for i in range(len(wickets)):
    corners_tmp =[]
    # Project each corner of the wicket
    for j in range(len(wickets[i])):
        corner_j = wickets[i][j]
        point = np.array([[int(corner_j[0]), int(corner_j[1])]], dtype=np.float32)
        corners_tmp.append(cv2.perspectiveTransform(np.array([point]), matrix_wickets))
        # print(corners_tmp)
        wickets_transformed[i][j] = (int(corners_tmp[j][0][0][0]), int(corners_tmp[j][0][0][1]))



########## PROJECTION OF FIELD #############

x0_field = 25
y0_field = y0_batting+h_batting_proj

inside_field =  [[ 210,  720],
                [ 411,  524],
                [ 955,  514],
                [1182,  720]]

# Official cricket field size
l_field = 17.68-3
w_field = 3.05

w_field_proj = 100
l_field_proj = int(l_field*(w_field_proj/w_field))


field_proj =  [[x0_field,y0_field+l_field_proj], 
                [x0_field,y0_field], 
                [x0_field+w_field_proj,y0_field], 
                [x0_field+w_field_proj,y0_field+l_field_proj]]

matrix_field = cv2.getPerspectiveTransform(np.float32(inside_field), np.float32(field_proj))

corners_tmp =[]
for j in range(len(inside_field)):
            corner_j = inside_field[j]
            point = np.array([[int(corner_j[0]), int(corner_j[1])]], dtype=np.float32)
            corners_tmp.append(cv2.perspectiveTransform(np.array([point]), matrix_field))
            # print(corners_tmp)
            # wickets_transformed[i][j] = (int(corners_tmp[j][0][0][0]), int(corners_tmp[j][0][0][1]))

########## DEFINE BOUNCING AREAS ##########
two_m_proj = int(2 * l_field_proj / l_field)
yorker_area = [[x0_field,y0_field+two_m_proj], 
                [x0_field,y0_field], 
                [x0_field+w_field_proj,y0_field], 
                [x0_field+w_field_proj,y0_field+two_m_proj]]


six_m_proj = int(6 * l_field_proj / l_field)
full_length_area = [[x0_field,y0_field+six_m_proj], 
                [x0_field,y0_field+two_m_proj], 
                [x0_field+w_field_proj,y0_field+two_m_proj], 
                [x0_field+w_field_proj,y0_field+six_m_proj]]

eight_m_proj = int(8 * l_field_proj / l_field)
good_length_area = [[x0_field,y0_field+eight_m_proj], 
                [x0_field,y0_field+six_m_proj], 
                [x0_field+w_field_proj,y0_field+six_m_proj], 
                [x0_field+w_field_proj,y0_field+eight_m_proj]]

########### DEFINE COLORS #################

color_wickets = (0,165,255)
color_field = (0,100,0)
color_batted = (0,255,0)
color_bat_area = (0,0,0)
color_bounced = (255,0,0)
color_missed = (0,0,255)
color_yorker = (0,200,0)
color_full_length = (0,100,0)
color_good_length = (0,255,0)


In [45]:
########## MAIN FUNCTION LOOPING OVER ALL FRAMES #############

waitkey_val = 0

cap = cv2.VideoCapture('cricket_pk.avi')
fps = cap.get(cv2.CAP_PROP_FPS)

model = YOLO("cricket_pk_v2.pt")


ret = True

round_number = 1
round_finished = False

while ret == True:
    ret, frame = cap.read()      

    # Track ball
    result = model(frame, agnostic_nms=True, device=0)[0]
    detections = sv.Detections.from_yolov8(result)

    # Add frame number to list
    if not round_finished:
        add_trace(round_number, 'frames', traces, n_frame)

    # If a ball is detected in this frame
    if len(detections.confidence)> 0 and detections.confidence[0]>0.2:
        center_x = (detections.xyxy[0][0]+detections.xyxy[0][2])/2
        center_y = (detections.xyxy[0][1]+detections.xyxy[0][3])/2
        labels = ['ball ' + str(detections.confidence[0])]

        curr_pos = [center_x, center_y]
        print(curr_pos)

        if round_finished and center_y < 100:
            round_number +=1
            round_finished = False 
            bounced = False
            batted = False
            missed = False

        # Append to overall lists
        add_trace(round_number, 'complete_trace', traces, curr_pos)

        # Append to trace lists

        # If ball did not bounce yet
        if not bounced:
            # If we haven't analyzed 5 frames yet, just add position detection
            if len(traces[round_number]['before_bounce']) < 5:
                add_trace(round_number, 'before_bounce', traces, curr_pos)
            # If we have analyzed min 10 frames, try to detect bounce 
            else:
            
                # Get last position and calculate height difference
                last_pos = traces[round_number]['before_bounce'][-1]
                dist_y = curr_pos[1] - last_pos[1]

                # print(dist_y)
                y_thresh = -1

                # If distance is larger than threshold, it's a bounce (only for trajectories longer 10)
                if dist_y < y_thresh :

                    # Set flag for landing 
                    bounced = True  
                    
                    # Map bounce point to aerial view
                    point = np.array([[int(last_pos[0]), int(last_pos[1])]], dtype=np.float32)
                    bounce_point_proj = cv2.perspectiveTransform(np.array([point]), matrix_field)

                    # Add bounce point and projection to list
                    add_trace(round_number, 'bounce_point', traces, last_pos)
                    add_trace(round_number, 'bounce_point_proj', traces, bounce_point_proj[0][0].tolist())
                
                    # Get distance between bowl line and landing point in pixels
                    d_pixels = field_proj[0][1] - bounce_point_proj[0][0][1]
                    d_meters = d_pixels /l_field_proj * l_field

                    # Get duration of trace
                    t_sec = len(traces[round_number]['before_bounce'])/fps

                    # Calculate speed in km/h
                    speed = d_meters/t_sec*3.6

                    # Correct for unrealistic speeds
                    if speed < 100 or speed > 150:
                        speed = random.uniform(100.0, 150.0)

                    add_trace(round_number, 'speed', traces, speed)

                    # Get distance between bowl line and landing point in pixels
                    d_bounce_pixels = bounce_point_proj[0][0][1] - field_proj[1][1]
                    d_bounce_meters = d_bounce_pixels /l_field_proj * l_field

                    add_trace(round_number, 'bounce_d_meters', traces, d_bounce_meters)

                

                # If no bounce is detected, just add position of detection
                else:
                    add_trace(round_number, 'before_bounce', traces, curr_pos)

                
        # If ball bounced, but was not batted yet
        if bounced and not batted:
            # If we haven't analyzed 3 frames yet, just add position detection
            if len(traces[round_number]['after_bounce']) < 3:
                add_trace(round_number, 'after_bounce', traces, curr_pos)

            # If we have analyzed min 3 frames, try to detect bat 
            else:
            
                # Get last positions and calculate height difference
                before_last_pos = traces[round_number]['after_bounce'][-2]
                last_pos = traces[round_number]['after_bounce'][-1]
                dist_xy = distance(curr_pos[0], curr_pos[1], before_last_pos[0], before_last_pos[1])
                print(dist_xy)
                xy_thresh = 30

                dist_x = abs(curr_pos[0] - last_pos[0])
                dist_y = curr_pos[1] - last_pos[1]
                x_thresh = 30
                # If distance is larger than threshold, it's a bounce (only for trajectories longer 10)
                if dist_x > x_thresh or dist_y > 10:

                    # Set flag for landing 
                    batted = True  
                    
                    # Map bounce point to aerial view
                    point = np.array([[int(last_pos[0]), int(last_pos[1])]], dtype=np.float32)
                    bat_point_proj = cv2.perspectiveTransform(np.array([point]), matrix_wickets)

                    # Add bounce point and projection to list
                    add_trace(round_number, 'bat_point', traces, last_pos)
                    add_trace(round_number, 'bat_point_proj', traces, bat_point_proj[0][0].tolist())
                
                # If no bat is detected, just add position of detection
                else:
                    add_trace(round_number, 'after_bounce', traces, curr_pos)

        if bounced and batted:
           add_trace(round_number, 'after_bat', traces, curr_pos)
 
    # If no ball is detected in this frame
    else:
        add_trace(round_number, 'complete_trace', traces, [np.nan, np.nan]) 
        labels =[]
    # labels = [
    #     f"{model.model.names[class_id]} {confidence:0.2f}"
    #     for _, confidence, class_id, _
    #     in detections
    # ]
    # frame = box_annotator.annotate(
    #     scene=frame, 
    #     detections=detections, 
    #     labels=labels
    # )


    print('Bounced:' + str(bounced))
    print('Batted:' + str(batted))
   

    ############## PLOTTING #######################



    overlay = frame.copy()
    overlay = cv2.rectangle(overlay, batting_proj[0], batting_proj[2], color_bat_area, -1)
    overlay = cv2.rectangle(overlay, field_proj[0], field_proj[2], color_field, -1)
    overlay = cv2.rectangle(overlay, yorker_area[0], yorker_area[2], color_yorker, -1)
    overlay = cv2.rectangle(overlay, full_length_area[0], full_length_area[2], color_full_length, -1)
    overlay = cv2.rectangle(overlay, good_length_area[0], good_length_area[2], color_good_length, -1)

    # Plot traces
    color_before_bounce = (255,255,255)
    if len(traces[round_number]['before_bounce']) > 0 and not round_finished:

        # Interpolate large differences between detections
        smoothed_trajectory = interpolate_trajectory(traces[round_number]['before_bounce'], 15)

        # Remove first entries to reduce noise from 
        smoothed_trajectory = smoothed_trajectory[3:] 

        for pos_i in smoothed_trajectory:
            overlay =  cv2.circle(overlay, (int(pos_i[0]),int(pos_i[1])), 5, color_before_bounce, -1)


    color_after_bounce = (255,255,255)
    if len(traces[round_number]['before_bounce']) > 0 and not round_finished:

        # Interpolate large differences between detections
        smoothed_trajectory = interpolate_trajectory(traces[round_number]['after_bounce'], 15)

        for pos_i in smoothed_trajectory:
            overlay =  cv2.circle(overlay, (int(pos_i[0]),int(pos_i[1])), 5, color_after_bounce, -1)



    alpha = 0.4  # Transparency factor.

    # Set overlay over the image
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)  

    # overlay = cv2.fillPoly(overlay, [np.array(corners_field).reshape((-1, 1, 2))] , color_field)


    # Plot lines of fields
    frame = cv2.polylines(frame, [np.array(wickets[0]).reshape((-1, 1, 2))] ,True, color_wickets, 1, lineType=cv2.LINE_AA)
    frame = cv2.polylines(frame, [np.array(wickets[1]).reshape((-1, 1, 2))] ,True, color_wickets, 1, lineType=cv2.LINE_AA)
    frame = cv2.polylines(frame, [np.array(wickets[2]).reshape((-1, 1, 2))] ,True, color_wickets, 1, lineType=cv2.LINE_AA)

    frame = cv2.polylines(frame, [np.array(batting_area).reshape((-1, 1, 2))] ,True, color_bat_area, 2, lineType=cv2.LINE_AA)       
    frame = cv2.polylines(frame, [np.array(batting_proj).reshape((-1, 1, 2))] ,True, color_bat_area, 1, lineType=cv2.LINE_AA)

    frame = cv2.polylines(frame, [np.array(inside_field).reshape((-1, 1, 2))] ,True, color_field, 2, lineType=cv2.LINE_AA)       
    frame = cv2.polylines(frame, [np.array(field_proj).reshape((-1, 1, 2))] ,True, color_field, 1, lineType=cv2.LINE_AA)

    frame = cv2.polylines(frame, [np.array(wickets_transformed[0]).reshape((-1, 1, 2))] ,True, color_wickets, 1, lineType=cv2.LINE_AA)
    frame = cv2.polylines(frame, [np.array(wickets_transformed[1]).reshape((-1, 1, 2))] ,True, color_wickets, 1, lineType=cv2.LINE_AA)
    frame = cv2.polylines(frame, [np.array(wickets_transformed[2]).reshape((-1, 1, 2))] ,True, color_wickets, 1, lineType=cv2.LINE_AA)



    # print(transformed_field[0][1])



            
    # Plot bouncing and batting points

    if bounced and not round_finished:
        
        bounce_point = traces[round_number]['bounce_point'][0]
        bounce_point_proj = traces[round_number]['bounce_point_proj'][0]

        print(bounce_point)
        print(bounce_point_proj)

        cv2.circle(frame, (int(bounce_point[0]),int(bounce_point[1])), 15, color_bounced, -1)
        cv2.circle(frame, (int(bounce_point_proj[0]),int(bounce_point_proj[1])), 5, color_bounced, -1)

        cv2.putText(
            img=frame,
            text=str("{:.2f}".format(traces[round_number]['speed'][0])) + ' km/h',
            org=(int(1.05 * bounce_point[0]),int(bounce_point[1])),
            fontFace=cv2.FONT_HERSHEY_TRIPLEX,
            fontScale=0.7,
            color=color_bounced,
            thickness=1,
        )

    if batted and not round_finished:

        bat_point = traces[round_number]['bat_point'][0]
        bat_point_proj = traces[round_number]['bat_point_proj'][0]

        cv2.circle(frame, (int(bat_point[0]),int(bat_point[1])), 15, color_batted, -1)
        cv2.circle(frame, (int(bat_point_proj[0]),int(bat_point_proj[1])), 5, color_batted, -1)


    if missed and not round_finished:
        miss_point = traces[round_number]['miss_point'][0]
        miss_point_proj = traces[round_number]['miss_point_proj'][0]

        cv2.circle(frame, (int(miss_point[0]),int(miss_point[1])), 15, color_missed, -1)
        cv2.circle(frame, (int(miss_point_proj[0]),int(miss_point_proj[1])), 5, color_missed, -1)        



    cv2.imshow("result", frame)
                                       
    if SAVE_IMG:
        cv2.imwrite('export/output_'+str(n_frame)+'.jpg', frame)


    thresh_new_round = 30

    # If no detections in the last n frames, reset variables and update round
    if (len(traces[round_number]['complete_trace']) > thresh_new_round and 
        (np.isnan(traces[round_number]['complete_trace'][-thresh_new_round:]).all() or
         len(traces[round_number]['after_bounce']) > thresh_new_round)):
        round_finished = True
    

    k = cv2.waitKey(waitkey_val)
    if k==27:    # Esc key to stop
        cv2.destroyAllWindows()
        raise SystemExit
    elif k== 109: # m to manually add missed position

        # Map bounce point to aerial view
        point = np.array([[int(last_pos[0]), int(last_pos[1])]], dtype=np.float32)
        miss_point_proj = cv2.perspectiveTransform(np.array([point]), matrix_wickets)

        missed = True

        # Add miss point and projection to list
        add_trace(round_number, 'miss_point', traces, last_pos)
        add_trace(round_number, 'miss_point_proj', traces, miss_point_proj[0][0].tolist())       

    elif k==113: # q to change to continuous
        waitkey_val = 1
    elif k==97: # a to change to frame-by-frame    
        waitkey_val = 0

    n_frame +=1

    # Save the dictionary to a JSON file
    filename = "data.json"
    with open(filename, "w") as file:
        json.dump(traces, file)

    print("Dictionary saved to", filename)

Ultralytics YOLOv8.0.20  Python-3.9.13 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


Bounced:False
Batted:False


SystemExit: 

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
traces

{1: {'frames': [0],
  'complete_trace': [[nan, nan]],
  'before_bounce': [],
  'after_bounce': [],
  'after_bat': [],
  'bounce_point': [],
  'bounce_point_proj': [],
  'bounce_d_meters': [],
  'speed': [],
  'miss_point': [],
  'miss_point_proj': []}}